In [1]:
#importing tensoflow
import tensorflow as tf

In [2]:
#importing Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_DATA/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
#initialising contants according our decided architecture of CNN
input_width = 28 #input image width
input_height = 28 #input image height
input_channels = 1 # No of Channels of input image
input_pixels = 784 # product of input image height and input image width

n_conv1 = 32 # no of cnn units in 1st Convolution layer
n_conv2 = 64 # no of cnn units int 2nd Convolution layer

stride_conv1 = 1 #Stride of filter in 1st Convolution layer
stride_conv2 = 1 # Stride of filter in 2nd Convolution layer

conv1_k = 5 # Dimension of Square filter in 1st Convoltuion layer
conv2_k = 5 # Dimension of Square filter in 2nd Convolution layer

max_pool1_k = 2 # Dimension of filter in 1 pooling layer
max_pool2_k = 2 # Dimension of filter in 2nd pooling layer

n_hidden = 1024 # No of units in hidden layer 
n_out = 10 # No of output unit, which is equal no of labels in the data

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) *n_conv2  # no of units of input to the hidden layer 

In [4]:
#initialising random weights and biases to all the filters and hidden layer

weights = {
    "wc1" : tf.Variable(tf.random_normal([conv1_k, conv1_k, input_channels, n_conv1])),
    "wc2" : tf.Variable(tf.random_normal([conv2_k, conv2_k, n_conv1, n_conv2])),
    "wh1" : tf.Variable(tf.random_normal([input_size_to_hidden, n_hidden])),
    "wo" : tf.Variable(tf.random_normal([n_hidden, n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bh1" : tf.Variable(tf.random_normal([n_hidden])),
    "bo" : tf.Variable(tf.random_normal([n_out])),
}

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
# Conv function which return output after each convolution operation in the neural net

def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x, weights, padding="SAME", strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

In [6]:
# maxpooling function which will return output after each pooling (max) operation in the neural net
def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding = "SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1])

In [7]:
# cnn funtion is the function which forward propogate the input through the convolution and pooling layers as per the decided architecture

def cnn(x, weights, biases, keep_prob):
    x = tf.reshape(x, shape = [-1 ,input_height, input_width, input_channels]) #reshaping the input data point to desired dimensions 
    
    #passing data through 1st convolution layer
    conv1 = conv(x, weights['wc1'], biases['bc1'], stride_conv1)
    #passing data through 1st pooling layer
    conv1_pool = maxpooling(conv1, max_pool1_k)
    
    
    #passing data through 2nd convolution layer
    conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    #passing data through 2nd pooling layer
    conv2_pool = maxpooling(conv2, max_pool2_k)
    
    #reshaping the output of 2nd pooling layer as the hidden layer requirement
    hidden_input = tf.reshape(conv2_pool, shape = [-1, input_size_to_hidden])
    
    #multiplying with weights and adding biases without passing it into activation function
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1'])
    
    #output of hidden layer after applying activation function
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    
    #final output of hidden layer after adding dropout layer in order to avoid overfitting
    hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob) 
   
    #final output of the neural net
    output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo'])
    return output

In [8]:
#defining placeholders for holding the input data and passing it into neural net

x = tf.placeholder("float", [None, input_pixels])
y = tf.placeholder(tf.int32, [None, n_out])
keep_prob = tf.placeholder("float")

#calling cnn to get output from neural net but this does not gives the final output, for final output we need to run a session
pred = cnn(x, weights, biases, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
#calculating the cost which is measure of how much our predicted labels are different from actual labels
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels = y))

In [10]:
# defining the optimizer which will backpropogate and reset weights of out neural net in order to minimise cost and having better predictions
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [11]:
# initialing a session and initialing all the variables which are defined above
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [12]:
#Running the epochs and session over neural net 
batch_size = 100
for i in range(20): # no of epochs
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost,optimize], feed_dict={x:batch_x , y:batch_y, keep_prob:0.8})
        total_cost += c
    print("epoch ", i+1, " completed")

epoch  1  completed
epoch  2  completed
epoch  3  completed
epoch  4  completed
epoch  5  completed
epoch  6  completed
epoch  7  completed
epoch  8  completed
epoch  9  completed
epoch  10  completed
epoch  11  completed
epoch  12  completed
epoch  13  completed
epoch  14  completed
epoch  15  completed
epoch  16  completed
epoch  17  completed
epoch  18  completed
epoch  19  completed
epoch  20  completed


In [13]:
#Getting the prediction from the trained neural net
predictions = tf.argmax(pred, 1)

#getting the actual label 
correct_labels = tf.argmax(y, 1)

#counting correct prediction from the model
correct_predictions = tf.equal(predictions, correct_labels)

#appling the above functions inside session
predictions,correct_preds  = sess.run([predictions, correct_predictions], feed_dict={x:mnist.test.images,
                                              y:mnist.test.labels, keep_prob:1.0})

print("Predicted correctlty ",correct_preds.sum(), " out of ", len(predictions))

Predicted correctlty  9859  out of  10000
